In [1]:
import deepsmiles
import itertools as it
import json
import mbuild as mb
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#import pymatgen as mg
#from pymatgen import symmetry

/home/chris/miniconda3/envs/mosdev/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


_ColormakerRegistry()

In [2]:
'''
THOUGHTS:

1. Build a dictionary for each compound
        - Name
        - Subclass (backbone, branch, constituent)
        - SMILES string
        - DeepSMILES string
        - Polyermization indices (DeepSMILES string index numbers)
        - Bonding indices
        - Chemical name
        - Mass (Later)
        - Symmetry (Later)
        - Chirality (Later)
        - ?


Dictionary: One created for each compound

'''

'\nTHOUGHTS:\n\n1. Build a dictionary for each compound\n        - Name\n        - Subclass (backbone, branch, constituent)\n        - SMILES string\n        - DeepSMILES string\n        - Polyermization indices (DeepSMILES string index numbers)\n        - Bonding indices\n        - Chemical name\n        - Mass (Later)\n        - Symmetry (Later)\n        - Chirality (Later)\n        - ?\n\n\nDictionary: One created for each compound\n\n'

In [2]:
def convert_smiles(smiles=False, deep=False):   
    '''
    smiles and deep must be str format
    Converts from SMILES to DeepSMILES and vice versa.
    Whichever has a string provided, will convert to the other.
    If strings are proivded for both, then nothing happens
    '''
    converter = deepsmiles.Converter(rings=True, branches=True)
    if smiles and deep:
        print('Only provide a string for one of smiles or deep')
        return()
    if smiles: # Convert from SMILES to DeepSMILES
        deep_string = converter.encode(smiles)
        return deep_string
    if deep: # Convert from DeepSMILES to SMILES
        smiles_string = converter.decode(deep)
        return smiles_string 

'c1sccc1CCCCCC'
'0-1234-5678910'

def poly_smiles_idx(monomer_string, polyindex, length=2, ftype='mol2', string_only=True,
                    energy_min=False, save=False, visualize=False):
    
    monomer_string_d = convert_smiles(smiles=monomer_string)
    
    
def poly_smiles(monomer_string, length=2, ftype = 'mol2', string_only=True,
                energy_min=False, save=False, visualize=False):
    
    # Find how many branch-brackets are required at polymerization site
    atom_count = 0
    bracket_count = 0
    for s in monomer_string:
        if s.isalpha():
            atom_count += 1    
        if s == ')':
            bracket_count += 1 
    if bracket_count == 0:
        brackets = ')' * atom_count
    elif bracket_count != 0:
        brackets = ')' * (atom_count - bracket_count)
       
    # Find index num of poly site on modified DEEP SMILES string
    monomer_list = list(monomer_string)
    if '*' not in monomer_list:
        return(print('ERROR: Identify the wanted polymerization site using *x*'))
    key_indices = [index for index, value in enumerate(monomer_list) if value == '*']  
    if len(key_indices) != 2:   # Checks for only a single given poly site
        return(print('ERROR: Select only one polymerization site using *x*'))
    if key_indices[1] - key_indices[0] != 2:   # Check that the * are surrounding only a single atom
        return(print('ERROR: Select only one polymerization site using *x*'))
    monomer_list[key_indices[1]] = '{}' + '{}'.format(brackets) # Create poly site+brackets to the right of the atom
    monomer_list.remove('*')
    template = ''.join(monomer_list)  # Monomer string with the needed {} and without second * in the string
    monomer_list.remove('{}' + '{}'.format(brackets))
    monomer = ''.join(monomer_list)  # Pure deepsmiles monomer string without {} or *
    # What is the difference between monomer and monomer_string without **??
    print(monomer)   
    # Loop & format polymer
    polymer = '{}'
    for i in range(0, length):
        if i == length - 1:
            polymer = polymer.format(monomer)
            break
        polymer = polymer.format(template)

    polymer_smiles = convert_smiles(deep = polymer)
    if string_only:
        return(polymer_smiles)
    compound = mb.load(polymer_smiles, smiles = True)
    if energy_min:
        compound.energy_minimize(steps = 25, forcefield='GAFF', algorithm = 'md')
    if visualize:
        compound.visualize().show()  
    if save:
        file_name = "comp_{}mer.{}".format(length, ftype)
        compound.save(file_name, overwrite = True)
    return compound, polymer_smiles

def viz(smiles_string=None, deep=False, energy_min = False, json_file=False):
    
    if json_file:
        with open('typed-components/{}'.format(json_file)) as jf:
            d = json.load(jf)
            smiles_string = d['smiles']
    else:
        if deep:
            smiles_string = convert_smiles(deep = smiles_string)    
    comp = mb.load(smiles_string, smiles = True)
    if energy_min:
        comp.energy_minimize(algorithm='md', steps=100)
    comp.visualize().show()

def find_bond_site(smiles_string, deep = True):
    '''    
    '''
    bonding_dicts = []  # List of dictionaries of each bonding site
    smiles_string_list = list(smiles_string)
    template_list = list(smiles_string)
    for index, char in enumerate(smiles_string_list):
        d = {}
        if char.lower() == 'c':  # The index belongs to a carbon site 'c'
            next_ind = index + 1
            try:
                if smiles_string_list[index + 1].isalpha():  # The following character in the string is an atom
                    index_to_change = index  # Change the original site
                    original_value = char
                    smiles_string_list[index_to_change] = original_value + 'N)'
                else:  # The following character is the SMILES string is a symbol or number
                    try:
                        while not smiles_string_list[next_ind].isalpha(): # Find the next alpha containing index
                            next_ind += 1
                        index_to_change = next_ind - 1
                    except:
                        index_to_change = -1  # The SMILES string ends with with a non-alpha value
            
                    original_value = smiles_string_list[index_to_change]  # Add the branch to the last non-alpha
                    smiles_string_list[index_to_change] = original_value + 'N)'
    
            except:
                index_to_change = index # The SMILES string ends with a 'c'
                original_value = char
                smiles_string_list[index_to_change] = original_value + 'N)'
                
            temp_string = ''.join(smiles_string_list)
            viz(temp_string)
            print('Add as possible bonding site? (y/n) or (exit)')  # QUESTION No. 1
            add = input()
            if add.lower() == 'yes' or add.lower() == 'y':
                template_list[index_to_change] = original_value + '{}'
                print('Classify bonding site as (1) branch, (2) polymerization, or (3) both') # QUESTION No. 2
                site_type = input()
                if site_type == '1':
                    bond_site_type = 'branch'
                elif site_type == '2':
                    bond_site_type = 'poly'
                    num_of_bonds = 1
                elif site_type == '3':
                    bond_site_type = 'both'
                if bond_site_type == 'branch' or bond_site_type == 'both':
                    print('How many bonds can be formed from this atom? (1, 2, 3, 4..etc)') # QUESTION No. 3
                    num_of_bonds = int(input())
                d['index'] = index_to_change
                d['type'] = bond_site_type
                d['num_bonds'] = num_of_bonds
                bonding_dicts.append(d)
            if add.lower() == 'exit':
                break
            smiles_string_list[index_to_change] = original_value
        else:
            pass
    template_molecule_string = ''.join(template_list)
    
    return bonding_dicts, template_molecule_string
       
    
def find_symmetry(string, deep = True):
    '''
    Given a chemical structure, find and classify it's symmetry.
    Groups, axes of symmetry, planes of symmetry, chirality, etc...
    '''
    if deep:
        smiles_string = convert_smiles(string)
    else:
        smiles_string = string
    compound = mb.load(smiles_string, smiles = True)  # mbuild compound
    
    
def add_component(name, structure_type, string, full_name = 'Not Provided', deep_smiles = False):
    '''
    Create a dictionary for a single compound/component using a given SMILES string representation.
    Compound can be created starting with either standard SMILES or DeepSMILES formatting 
    Ultimately, the dictionary will contain both types of SMILES strings. 
    If the structure_type (subclass) is backbone, then the user will go through process of
    adding and typing bonding sites. Dictionary stores the bonding information.
    
    name : str, required
        Generic, identifiable name (Ex. "ITIC", "PTB7", "Benzene")
    string : str, required
        A SMILES string in either standard SMILES or DeepSMILES
    structure_type : str, required
        Options are backbone, branch, constituent. 
    full_name : str, optional, default = "Not Provided"
        Stores the full/official name of the structure
    deep_smiles : bool, required, default = False
        If string parameter is in the DeepSMILES format, then deep_smiles should be True
        If string parameter is in the standard SMILES format, then deep_smiles should be False
    
    TODO:
    Add try/except for building compound given the string/deep_smiles combo
    Add write dictionary to JSON file - Ask Mike about the Sig'c1ccc(cc1)S(=O)(=O)c%20cc(c2ccc(cc2)S(=O)(=O)c%19cc(c3ccc(cc3)S(=O)(=O)c%18cc(c4ccc(cc4)S(=O)(=O)c%17cc(c5ccc(cc5)S(=O)(=O)c%16cc(c6ccc(cc6)S(=O)(=O)c%15cc(c7ccc(cc7)S(=O)(=O)c%14cc(c8ccc(cc8)S(=O)(=O)c%13cc(c9ccc(cc9)S(=O)(=O)c%12cc(c%10ccc(cc%10)S(=O)(=O)c%11ccccc%11)ccc%12)ccc%13)ccc%14)ccc%15)ccc%16)ccc%17)ccc%18)ccc%19)ccc%20'nac .json file formats. Dictionary of dictionaries?
    
    ''' 
    d = {}
    if deep_smiles:  # string parameter is in DeepSMILES format
        smiles_string = convert_smiles(deep = string) 
        deep_smiles_string = string
    else:  #  string parameter is in standard SMILES format
        smiles_string = string
        deep_smiles_string = convert_smiles(smiles = string)
    if structure_type == 'backbone':
        bond_dict, template = find_bond_site(deep_smiles_string)
        d['template'] = template
    else:
        bond_dict = {}      
    
    d['name'] = name
    d['subclass'] = structure_type
    d['smiles'] = smiles_string
    d['deep_smiles'] = deep_smiles_string
    d['bonding'] = bond_dict
    d['full name'] = full_name
    #d['chiral']  Add later
    #d['symmetry']  Add later
    file_name = 'typed-components/{}.json'.format(name)
    with open(file_name, 'w') as fp:
        json.dump(d, fp)
    return d

def read_comp(fpath):
    
    with open(fpath) as jf:
        d = json.load(jf)
        for key in d:
            print('{}: {}'.format(key, d[key]))
    

In [21]:
#viz('Oc1ccc(cc1)S(=O)(=O)c1ccc(cc1)O')
viz('c1ccc(cc1)S(=O)(=O)c1ccc(cc1)')
deep_PES = convert_smiles(smiles = 'c3(c1ccc(cc1)S(=O)(=O)c2ccccc2)ccc(cc3)S(=O)(=O)c4ccccc4')
print(deep_PES)

/home/chris/miniconda3/envs/mosdef-smiles/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1ccc(cc1)S(=O)(=O)c1ccccc1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

ccccccc6))S=O)=O)cccccc6)))))))))))ccccc6))S=O)=O)cccccc6


In [35]:
poly_smiles('cccccc6))S=O)=O)cc*c*ccc6))',length=10, string_only = False, visualize=True, save=True)

cccccc6))S=O)=O)cccccc6))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

(< 232 particles, non-periodic, 251 bonds, id: 139919142772688>,
 'c1ccc(cc1)S(=O)(=O)c%20cc(c2ccc(cc2)S(=O)(=O)c%19cc(c3ccc(cc3)S(=O)(=O)c%18cc(c4ccc(cc4)S(=O)(=O)c%17cc(c5ccc(cc5)S(=O)(=O)c%16cc(c6ccc(cc6)S(=O)(=O)c%15cc(c7ccc(cc7)S(=O)(=O)c%14cc(c8ccc(cc8)S(=O)(=O)c%13cc(c9ccc(cc9)S(=O)(=O)c%12cc(c%10ccc(cc%10)S(=O)(=O)c%11ccccc%11)ccc%12)ccc%13)ccc%14)ccc%15)ccc%16)ccc%17)ccc%18)ccc%19)ccc%20')

In [34]:
PES10mer = 'Oc1ccc(cc1)S(=O)(=O)c%20cc(c2ccc(cc2)S(=O)(=O)c%19cc(c3ccc(cc3)S(=O)(=O)c%18cc(c4ccc(cc4)S(=O)(=O)c%17cc(c5ccc(cc5)S(=O)(=O)c%16cc(c6ccc(cc6)S(=O)(=O)c%15cc(c7ccc(cc7)S(=O)(=O)c%14cc(c8ccc(cc8)S(=O)(=O)c%13cc(c9ccc(cc9)S(=O)(=O)c%12cc(c%10ccc(cc%10)S(=O)(=O)c%11ccccc%11)ccc%12)ccc%13)ccc%14)ccc%15)ccc%16)ccc%17)ccc%18)ccc%19)ccc%20O'
viz(PES10mer)

/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule Oc1ccc(cc1)S(=O)(=O)c1cc(c2ccc(cc2)S(=O)(=O)c2cc(c3ccc(cc3)S(=O)(=O)c3cc(c4ccc(cc4)S(=O)(=O)c4cc(c5ccc(cc5)S(=O)(=O)c5cc(c6ccc(cc6)S(=O)(=O)c6cc(c7ccc(cc7)S(=O)(=O)c7cc(c8ccc(cc8)S(=O)(=O)c8cc(c9ccc(cc9)S(=O)(=O)c9cc(c%10ccc(cc%10)S(=O)(=O)c%10ccccc%10)ccc9)ccc8)ccc7)ccc6)ccc5)ccc4)ccc3)ccc2)ccc1O	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [104]:
branch = add_component(name = 'benzene-cyclopentane', structure_type='branch', string = "C=c2c(=O)c1c(cccc1)c2=C(CC)CC")

In [106]:
branch = 'ccc'
smi_branch = convert_smiles(deep = branch)
#print(smi_branch)
#viz(json_file = 'benzylehexane.json')
#viz(branch)
#viz(smi_branch, deep=False)

In [190]:
'''
class Compound:
    
    
    def __init__(self, backbone, branches, fill, poly):   
        self.fname = 'typed-components/{}.json'.format(backbone)
        self.fill = fill
        self.polymerize = poly
        self.branches = branches
        
        #self.smiles, self.deep_smiles = get_smiles_string(self.fname)

    def get_smiles(self):
        with open(self.fname) as jf:
            d = json.load(jf)
            smiles = d['smiles']
        self.smiles = smiles
    
    def smiles(self):
        with open(self.fname) as jf:
            d = json.load(jf)
            smiles = d['smiles']
            deep_smiles = d['deep_smiles']
        return smiles
        
    def get_branches(self):
        pass
    
    def build(self):
        pass
    
    def polyermize(self):
        pass
'''

In [8]:
def get_smiles(component, type='smiles'):
    
    with open('typed-components/{}.json'.format(component)) as jf:
        d = json.load(jf)
        if type == 'smiles':
            smiles = d['smiles']
        elif type == 'deep_smiles':
            smiles = d['deep_smiles']
        elif type == 'template':
            smiles = d['template']
        return smiles

In [25]:
def build_compound(backbone, branches, polymerize=False):
    
    def get_bonding_sites():
        bond_indices = []
        with open('typed-components/{}.json'.format(backbone)) as jf:
            d = json.load(jf)
            bond_info = d['bonding']
            for dictionary in bond_info:
                if not polymerize:
                    bond_indices.append(dictionary['index'])
                elif polyermize:
                    if dictionary['type'] != 'poly':
                        bond_indices.append(dictionary['index'])
        return bond_indices
        
    backbone_smiles = get_smiles(backbone)
    backbone_deep_smiles = get_smiles(backbone, type='deep_smiles')
    backbone_template = get_smiles(backbone, type = 'template')
    branch_smiles = [get_smiles(branch) for branch in branches]
    branch_deep_smiles = [get_smiles(branch, type = 'deep_smiles') for branch in branches]
    bond_indices = get_bonding_sites()
    
    print(backbone_smiles)
    print(backbone_deep_smiles)
    print(backbone_template)
    print(branch_smiles)
    print(branch_deep_smiles)
    print(bond_indices)
    
        def assign_branches():
            
    
    return bond_indices, branch_smiles

In [26]:
backbone = 'itic'
branches = ['benzene-cyclopentane', 'benzylehexane']
bonding_sites, branch_smiles = build_compound(backbone, branches)

C6c4cc3c2sc1ccsc1c2Cc3cc4c7sc5ccsc5c67
Cccccscccsc5c8Cc%11cc%15cscccsc5c%248
C{}cc{}ccscc{}c{}sc5c8C{}c%11c{}c%15cscc{}c{}sc5c%248
['C=c2c(=O)c1c(cccc1)c2=C(CC)CC', 'c1ccc(CCCCCC)cc1']
['C=cc=O)cccccc6))))c5=CCC))CC', 'ccccCCCCCC))))))cc6']
[0, 2, 7, 8, 14, 19, 27, 28]


In [5]:
p3ht = 'c1sccc1CCCCCC'
viz(p3ht, deep=False)

/home/chris/miniconda3/envs/mosdef-smiles/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1sccc1CCCCCC	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
convert_smiles(smiles = p3ht)

NameError: name 'p3ht' is not defined

In [13]:
poly_smi = poly_smiles(monomer_string='cs*c*cc5CCCCCC', length=10)
print(poly_smi)

csccc5CCCCCC
c%10sc(c9sc(c8sc(c7sc(c6sc(c5sc(c4sc(c3sc(c2sc(c1sccc1CCCCCC)cc2CCCCCC)cc3CCCCCC)cc4CCCCCC)cc5CCCCCC)cc6CCCCCC)cc7CCCCCC)cc8CCCCCC)cc9CCCCCC)cc%10CCCCCC


In [14]:
'''
p3ht_comp = mb.load(p3ht, smiles=True)
for p in p3ht_comp.particles():
    print(p)
#         self.add(mb.Port(anchor=carbon, orientation=[0, 1, 0], separation=0.075), 'up')
'''

"\np3ht_comp = mb.load(p3ht, smiles=True)\nfor p in p3ht_comp.particles():\n    print(p)\n#         self.add(mb.Port(anchor=carbon, orientation=[0, 1, 0], separation=0.075), 'up')\n"

In [15]:
'''
p3ht_comp = mb.load(p3ht, smiles=True)
p3ht_comp.add(mb.Port(anchor=p3ht_comp[2], orientation=[0, -1, 0], separation=0.2,), 'down')
p3ht_comp.add(mb.Port(anchor=p3ht_comp[0], orientation=[0, 1, 0], separation=0.2,), 'up')

p3ht_dimer = mb.recipes.Polymer(p3ht_comp, n=2)
'''

"\np3ht_comp = mb.load(p3ht, smiles=True)\np3ht_comp.add(mb.Port(anchor=p3ht_comp[2], orientation=[0, -1, 0], separation=0.2,), 'down')\np3ht_comp.add(mb.Port(anchor=p3ht_comp[0], orientation=[0, 1, 0], separation=0.2,), 'up')\n\np3ht_dimer = mb.recipes.Polymer(p3ht_comp, n=2)\n"

In [16]:
p3ht1 = 'c1sccc1(N)'
p3ht2 = 'c1scc(N)c1'
viz(p3ht1, deep=False)
viz(p3ht2, deep=False)

/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1sccc1N	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1scc(N)c1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
p3ht = 'c1sccc1CCCC'
p3ht_d = convert_smiles(smiles=p3ht)
print(p3ht_d)

csccc5CCCC


In [ ]:
zz